In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt

In [2]:
sensor_names=[f's_{i}' for i in range(1,22)]
settings=['set_1','set_2','set_3']
cols=['unit_nr','time_cycles']+settings+sensor_names
train=pd.read_csv(('CMAPSSData/train_FD001.txt'),sep='\s+',header=None,names=cols) # change it here
test=pd.read_csv(('CMAPSSData/test_FD001.txt'),sep='\s+',header=None,names=cols)#change it here
actual=pd.read_csv(('CMAPSSData/RUL_FD001.txt'),sep='\s+',header=None,names=['RUL'])#change it here
actual=actual.to_numpy().flatten().tolist()

In [3]:
max_cycle_train=[]
for i in range(1,101):#change it here
    row=train.loc[train['unit_nr']==i]
    max_cycle_train.append(row['time_cycles'].max())

max_cycle_test=[]
for i in range(100):#change it here
    row=test.loc[test['unit_nr']==i+1]['time_cycles']
    max_cycle_test.append(len(row)+int(actual[i]))

In [4]:
def add_rul(data,max_cycle):
    RUL=[]
    for i,j in enumerate(max_cycle):
        row=data.loc[data['unit_nr']==i+1]['time_cycles']
        for k in row:
            RUL.append(j-k)
    return RUL

train_rul=pd.DataFrame(add_rul(train,max_cycle_train))
train['RUL']=train_rul
test['RUL']=(add_rul(test,max_cycle_test))

In [5]:
drop_sensors = ['s_1','s_5','s_6','s_10','s_16','s_18','s_19']
index_names=['time_cycles']
drop_labels = index_names+['set_1','set_2','set_3']+drop_sensors
X_train = train.drop(drop_labels, axis=1)
X_test = test.drop(drop_labels, axis=1)

In [6]:
col_to_scale=X_train.columns.to_list()
col_to_scale.pop(0)
col_to_scale.pop(-1)

'RUL'

In [7]:
train['RUL']=train_rul.clip(upper=125)

In [8]:
def exponential_smoothing(df, sensors, alpha=0.4):
    cf=[]
    for i in sensors:
        C=df[i].copy()
        X=[C[0]]
        for i in range(1,len(C)):
            X.append(alpha * C[i-1]+(1-alpha)*X[i-1])
        cf.append(X)
    return np.array(cf)
        
e_train=exponential_smoothing(X_train,col_to_scale,0.3)
e_test=exponential_smoothing(X_test,col_to_scale,0.3)
ex_train=X_train.copy()
ex_test=X_test.copy()
for i,j in enumerate(col_to_scale):
    ex_train[j]=e_train[i]
for i,j in enumerate(col_to_scale):
    ex_test[j]=e_test[i]

In [9]:
scaler=MinMaxScaler()
def scaling(x_scale):
    x_scale[col_to_scale]=scaler.fit_transform(x_scale[col_to_scale])
    return x_scale
ex_train=scaling(ex_train)
ex_test=scaling(ex_test)

In [10]:
new_test=[]
for i in range(1,101): #change it here
    df=ex_test.loc[ex_test['unit_nr']==i]
    new_test.append(df.iloc[-25:,1:15].to_numpy())

In [11]:
new_test=np.array(new_test).reshape(100,25,14)#change it here

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences as pad
def sequences(x,window=15):
    M=[]
    N=[]
    cnt=[]
    for i in range(1,101):#change it here
        sub_train=x.loc[x['unit_nr']==i]
        array=sub_train.to_numpy()
        X=[]
        Y=[]
        count=0
        label=list()
        for j in range(len(array)-window):
            row=[a for a in array[j:j+window,1:15]]
            X.append(row)
            label=array[j+window,15:16]
            Y.append(label)
            count+=1
        for j in range(len(array)-window,len(array)):
            row=[a for a in array[j:len(array),1:15]]
            X.append(row)
            m=array[len(array)-1,15:16]
            Y.append([-100])
        cnt.append(count)
        padded_sequences = pad(X, padding='post', dtype='float32',value=-100)
        M.append(padded_sequences)
        N.append(Y)
    X_sequences=pad(M, padding='post', dtype='float32',value=-100)
    Y_sequences=pad(N, padding='post', dtype='float32',value=-100)
    return np.array(X_sequences),np.array(Y_sequences),cnt

seq_x_train,seq_y_train,count1=sequences(ex_train,25)

In [13]:
from sklearn.model_selection import train_test_split
x1,x2,y1,y2=train_test_split(seq_x_train,seq_y_train,test_size=0.2,shuffle=True,random_state=42)


In [14]:
func= lambda i: i.reshape(i.shape[0]*i.shape[1],i.shape[2],i.shape[3])
x1=func(x1)
x2=func(x2)
y1=y1.flatten()
y2=y2.flatten()

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Masking,Dense,Dropout,LeakyReLU
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import R2Score
from tensorflow.keras.optimizers import Adam

model1 = Sequential()
model1.add(Masking(mask_value=-100,input_shape=(25,14)))
model1.add(Dropout(0.3))
model1.add(LSTM(128,activation='sigmoid',return_sequences=True))
model1.add(Dropout(0.3))
model1.add(LSTM(128,activation='sigmoid'))
model1.add(Dense(8,LeakyReLU(0.3)))
model1.add(Dense(1, 'linear'))
model1.summary()

C:\Users\indra\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 25, 14)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 25, 14)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 25, 128)        │        73,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 25, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │         1,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 205,841 (804.07 KB)

 Trainable params: 205,841 (804.07 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), metrics=[R2Score()])

In [18]:
model1.fit(x1, y1, validation_data=(x2, y2), epochs=10)

Epoch 1/10
905/905 ━━━━━━━━━━━━━━━━━━━━ 25s 24ms/step - loss: 5109.3032 - r2_score: 0.5596 - val_loss: 1106.1809 - val_r2_score: 0.9003
Epoch 2/10
905/905 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - loss: 1203.1154 - r2_score: 0.8965 - val_loss: 928.6794 - val_r2_score: 0.9163
Epoch 3/10
905/905 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - loss: 1151.8481 - r2_score: 0.9010 - val_loss: 807.0703 - val_r2_score: 0.9273
Epoch 4/10
905/905 ━━━━━━━━━━━━━━━━━━━━ 21s 24ms/step - loss: 1122.3148 - r2_score: 0.9035 - val_loss: 781.6328 - val_r2_score: 0.9296
Epoch 5/10
905/905 ━━━━━━━━━━━━━━━━━━━━ 22s 24ms/step - loss: 1192.7701 - r2_score: 0.8995 - val_loss: 606.7197 - val_r2_score: 0.9453
Epoch 6/10
905/905 ━━━━━━━━━━━━━━━━━━━━ 22s 24ms/step - loss: 1111.1271 - r2_score: 0.9053 - val_loss: 712.8442 - val_r2_score: 0.9358
Epoch 7/10
905/905 ━━━━━━━━━━━━━━━━━━━━ 22s 25ms/step - loss: 1109.2947 - r2_score: 0.9055 - val_loss: 788.0065 - val_r2_score: 0.9290
Epoch 8/10
905/905 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step 

In [19]:
result=model1.predict(new_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step


In [20]:
from sklearn.metrics import r2_score

In [21]:
def evaluate(y_true, y_hat, label='test'):
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    print(f'set RMSE:{rmse}')
    r2=r2_score(y_true,y_hat)
    print(r2)
    
evaluate(actual,result.round())

set RMSE:36.46052660069517
0.23018503189086914
